In [ ]:
import pandas as pd
import matplotlib as plit
import sys
import os
sys.path.append(os.path.join("estadisticas_jupyter.ipynb", '..')) #Preguntar
from rutas import archivo_imagenes_etiquetadas_csv
from rutas import archivo_logs_csv

df=pd.read_csv(archivo_imagenes_etiquetadas_csv)
df_logs=pd.read_csv(archivo_logs_csv)
df

In [ ]:
#item 8
df.groupby('ultimo_perfil').sum()['tamaño']
#me tengo que traer los nombres de todos los usuarios y hacer un merge

In [ ]:
#item 7
df2=df.copy()
df2['tags']=df['tags'].str.split(',')
df2['tags'].explode()